## Data Cleaning

### Day-Ahead Price

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import csv

In [2]:
df_p_19_1 = pd.read_xml('raw/day_ahead_price/price_2019_1.xml', xpath=".//ns:Point", namespaces={'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'})
df_p_19_2 = pd.read_xml('raw/day_ahead_price/price_2019_2.xml', xpath=".//ns:Point", namespaces={'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'})
df_p_19_3 = pd.read_xml('raw/day_ahead_price/price_2019_3.xml', xpath=".//ns:Point", namespaces={'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'})
df_p_19_4 = pd.read_xml('raw/day_ahead_price/price_2019_4.xml', xpath=".//ns:Point", namespaces={'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'})

In [5]:
def p_c(df):
    
    # Identify cycles by detecting where 'position' restarts
    df['group'] = (df['position'] == 1).cumsum() - 1
    
    # Create a DataFrame with the complete range of positions from 1 to 24, repeated for each cycle (group)
    num_groups = df['group'].max() + 1
    print('num of groups:', num_groups)
    full_positions = pd.DataFrame({
        'position': list(range(1, 25)) * num_groups,
        'group': np.repeat(range(num_groups), 24)
    })
    
    # Merge the original DataFrame with the full positions to ensure all positions are included in each group
    df_fixed = full_positions.merge(df, on=['position', 'group'], how='left')
    
    # Forward fill missing 'price' values within each group
    df_fixed['price'] = df_fixed.groupby('group')['price.amount'].ffill()
    
    # Drop the 'group' column if it's not needed anymore
    df_fixed = df_fixed.drop(columns=['group'])
    return df_fixed

In [290]:
def read_price_xml(year):
    data_xml = pd.DataFrame()
    for i in range(1,5):
       df = pd.read_xml(f'raw/day_ahead_price/price_{year}_{i}.xml', xpath=".//ns:Point", namespaces={'ns': 'urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3'}) 
       data_xml = pd.concat([data_xml,df], axis=0, ignore_index=True) 
    return data_xml

In [352]:
def read_price(start_year = 2019, end_year = 2019):
    df_xml = read_price_xml(2019)
    df= p_c(df_xml)
    print(df.head())
    # if start_year == 2019:
    #     # df.loc['shift1']= df['price'].shift(-1)
    #     x = df['price'].shift(-1)
    #     df.loc['2019-04-01 00:00:00':, 'price'] = data.loc['2019-04-01 00:00:00':, 'shift1']
    return df

In [354]:
df_19_t = read_price(2019)

num of groups: 365
   position  price.amount  price
0         1         68.92  68.92
1         2         64.98  64.98
2         3         60.27  60.27
3         4         49.97  49.97
4         5         47.66  47.66


In [355]:
df_19_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   position      8760 non-null   int64  
 1   price.amount  8679 non-null   float64
 2   price         8760 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 205.4 KB


In [97]:
df_fin = pd.concat([df_19_fixed,df_20_fixed, df_21_fixed, df_22_fixed, df_23_fixed], axis=0, ignore_index=True) 

In [98]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   position      43824 non-null  int64  
 1   price.amount  43500 non-null  float64
 2   price         43824 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 1.0 MB


In [99]:
# Generate an hourly DatetimeIndex from 2019-01-01 to 2021-12-31
date_index = pd.date_range(start="2019-01-01 01:00:00", end="2024-01-01 00:00:00", freq="h")

# Set the generated DatetimeIndex as the index for the DataFrame
df_fin.index = date_index

In [100]:
df_fin.tail()

,position,price.amount,price
2023-12-31 20:00:00,20,39.64,39.64
2023-12-31 21:00:00,21,34.89,34.89
2023-12-31 22:00:00,22,13.30,13.30
2023-12-31 23:00:00,23,10.68,10.68
2024-01-01 00:00:00,24,3.17,3.17


In [105]:
df_fin.loc['2022-05-20']

,position,price.amount,price
2022-05-20 00:00:00,24,213.31,213.31
2022-05-20 01:00:00,1,211.10,211.10
2022-05-20 02:00:00,2,195.98,195.98
2022-05-20 03:00:00,3,189.56,189.56
2022-05-20 04:00:00,4,188.00,188.00
2022-05-20 05:00:00,5,189.56,189.56
2022-05-20 06:00:00,6,208.62,208.62
2022-05-20 07:00:00,7,236.04,236.04
2022-05-20 08:00:00,8,263.32,263.32
2022-05-20 09:00:00,9,250.00,250.00


In [101]:
df_fin.to_pickle('output/df_price_19_23.pkl')

TODO: 23data

- {'start': '2022-03-26T23:00Z', 'end': '2022-03-27T22:00Z'}
- {'start': '2022-10-29T22:00Z', 'end': '2022-10-30T23:00Z'}
- {'start': '2023-03-25T23:00Z', 'end': '2023-03-26T22:00Z'}
- {'start': '2023-10-28T22:00Z', 'end': '2023-10-29T23:00Z'}
- {'start': '2024-03-30T23:00Z', 'end': '2024-03-31T22:00Z'}

### Forecast Energy Generation

#### Solar B16

In [106]:
date_series_19 = pd.date_range(start='2019-01-01 01:00', end='2020-01-01 00:00', freq='1h')
date_series_20 = pd.date_range(start='2020-01-01 01:00', end='2021-01-01 00:00', freq='1h')
date_series_21 = pd.date_range(start='2021-01-01 01:00', end='2022-01-01 00:00', freq='1h')
date_series_22 = pd.date_range(start='2022-01-01 01:00', end='2023-01-01 00:00', freq='1h')
date_series_23 = pd.date_range(start='2023-01-01 01:00', end='2024-01-01 00:00', freq='1h')

In [5]:
# t = date_series_23[date_series_23 < pd.to_datetime('2023-11-14') and date_series_23 > pd.to_datetime('2023-11-14')]
# t = date_series_23[(date_series_23 < pd.to_datetime('2023-11-13 23:30:00')) | (date_series_23 > pd.to_datetime('2023-11-14 23:30:00'))]

In [108]:
def min2h(df_xml, date_series):
    df = df_xml.copy()
    df.drop('position', axis = 1, inplace = True)
    # df.rename(columns={'quantity': col_name}, inplace=True)
    df_hourly_data = df.groupby(df.index // 4).sum()
    df_hourly_data['Date'] = date_series
    df_hourly_data.set_index('Date', inplace= True)
    # df_hourly_data.rename(columns={'quantity': 'Alpha'}, inplace=True)
    return df_hourly_data

In [60]:
# def min2h_23(df_xml, date_series):
#     df = df_xml.copy()
#     df.drop('position', axis = 1, inplace = True)
#     # df.rename(columns={'quantity': col_name}, inplace=True)
#     df_hourly_data = df.groupby(df.index // 4).sum()
#     # Remove the specific date
#     # filtered_date_series = date_series[date_series < pd.to_datetime('2023-11-14') and date_series > pd.to_datetime('2023-11-14')]
#     filtered_date_series = date_series_23[(date_series_23 < pd.to_datetime('2023-11-13 23:30:00')) | (date_series_23 > pd.to_datetime('2023-11-14 23:30:00'))]
#     df_hourly_data['Date'] = filtered_date_series
#     df_hourly_data.set_index('Date', inplace= True)
#     return df_hourly_data

In [109]:
def missing_1114(df):
    
    df_1113 = df[df.index.date == pd.to_datetime('2023-11-13').date()]
    df_1115 = df[df.index.date == pd.to_datetime('2023-11-15').date()]
    df_1114_avg= (df_1113['quantity'].values +  df_1115['quantity'].values) / 2
    df_1114_avg = np.array([int(x) for x in df_1114_avg])
    df_1114_avg = df_1114_avg.reshape(24,1)
    df[df.index.date == pd.to_datetime('2023-11-14').date()] = df_1114_avg
    return df

## Energy Generation

In [168]:
def gene(folder,start_year, end_year):
    data = pd.DataFrame()
    for year in range(start_year, end_year +1 ):
        print(year)
        date_series = pd.date_range(start=f'20{year}-01-01 01:00', end=f'20{year+1}-01-01 00:00', freq='1h')
        # print(date_series[-5:])
        df_xml = pd.read_xml(f'raw/day_ahead_fore/{folder}/{folder}_{year}.xml', xpath=".//ns:Point", namespaces={'ns': 'urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0'})
        # print(df_xml.tail())
        df_hourly = min2h(df_xml, date_series)
        # print(df_hourly.tail())
        if year == 23 and folder != 'total_load':
            print('Fill missing 2023-11-14')
            df_hourly = missing_1114(df_hourly)
        data = pd.concat([data,df_hourly], axis=0) 
    data.to_pickle(f'output/df_{folder}_19_23.pkl')
    return data

In [169]:
tl = gene('total_load',19, 23)

19
20
21
22
23


In [170]:
tl.loc['2021-12-30']

,quantity
Date,
2021-12-30 00:00:00,49938
2021-12-30 01:00:00,47361
2021-12-30 02:00:00,45772
2021-12-30 03:00:00,44777
2021-12-30 04:00:00,44227
2021-12-30 05:00:00,44464
2021-12-30 06:00:00,46233
2021-12-30 07:00:00,50336
2021-12-30 08:00:00,57302


## All

In [232]:
df_price = pd.read_pickle('output/df_price_19_23.pkl')
df_price.drop(columns= ['position', 'price.amount'], inplace = True)
df_solar = pd.read_pickle('output/df_solar_19_23.pkl')
df_onshore = pd.read_pickle('output/df_onshore_19_23.pkl')
df_offshore = pd.read_pickle('output/df_offshore_19_23.pkl')
df_total_load = pd.read_pickle('output/df_total_load_19_23.pkl')

In [233]:
df_price.columns = ['price']
df_solar.columns = ['solar']
df_onshore.columns = ['wind_onshore']
df_offshore.columns = ['wind_offshore']
df_total_load.columns = ['total_load']

In [234]:
df_fore = pd.concat([df_price, df_solar, df_onshore, df_offshore, df_total_load], axis=1)

In [235]:
df_fore.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43824 entries, 2019-01-01 01:00:00 to 2024-01-01 00:00:00
Freq: h
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          43824 non-null  float64
 1   solar          43824 non-null  int64  
 2   wind_onshore   43824 non-null  int64  
 3   wind_offshore  43824 non-null  int64  
 4   total_load     43824 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 2.0 MB


In [225]:
df_fore.loc['2019-03-31 21:00:00': '2019-04-01 03:00:00']

,price,solar,wind_onshore,wind_offshore,total_load
2019-03-31 01:00:00,40.10,0,1521,807,32441
2019-03-31 02:00:00,37.33,0,1897,1305,31694
2019-03-31 03:00:00,40.03,0,2412,1629,31484
2019-03-31 04:00:00,37.38,0,2800,1963,31737
2019-03-31 05:00:00,33.67,0,3019,2218,32466
2019-03-31 06:00:00,32.00,58,3292,2358,33381
2019-03-31 07:00:00,33.98,420,3531,2468,34324
2019-03-31 08:00:00,33.07,1307,3822,2400,35207
2019-03-31 09:00:00,36.28,2555,4108,2329,36045
2019-03-31 10:00:00,37.42,3794,3982,2272,36068


In [222]:
df_fore.loc['2019-10-27 01:00:00': '2019-10-27 06:00:00']

,price,solar,wind_onshore,wind_offshore,total_load
2019-10-27 01:00:00,32.21,0,4547,2822,37198
2019-10-27 02:00:00,27.11,0,4284,2808,36267
2019-10-27 03:00:00,25.00,0,4225,2782,36267
2019-10-27 04:00:00,25.70,0,3935,2771,35908
2019-10-27 05:00:00,22.86,0,3601,2750,35824
2019-10-27 06:00:00,16.47,17,3201,2624,36011


In [236]:
df_fore.to_pickle('output/df_22_23.pkl')

In [ ]:
# 2019: Sun, Mar 31, 2019 – Sun, Oct 27, 2019

## cleaning

In [358]:
data = pd.read_pickle('output/df_22_23.pkl')

In [359]:
data.head()

,price,solar,wind_onshore,wind_offshore,total_load
2019-01-01 01:00:00,68.92,0,6723,2372,49563
2019-01-01 02:00:00,64.98,0,7094,2409,47240
2019-01-01 03:00:00,60.27,0,7569,2487,45642
2019-01-01 04:00:00,49.97,0,8012,2581,45236
2019-01-01 05:00:00,47.66,0,8207,2639,45681


In [360]:
def price_cleaner(info):
    data.loc[:,'shift1'] = data['price'].shift(-1)
    for start_date, end_date, value in info:
        print(start_date, end_date)
        data.loc[start_date : end_date, 'price'] = data.loc[start_date : end_date, 'shift1']
        data.loc[end_date, 'price'] = value
    data.drop(columns = ['shift1'],inplace = True)
    return data

In [361]:
info = [
    # start, dat light saving +1 day, 
    ['2019-04-01 00:00:00', '2019-10-28 00:00:00', 25.82],
    ['2020-03-30 00:00:00', '2020-10-26 00:00:00', 34.75],
    ['2021-03-29 00:00:00', '2021-11-01 00:00:00', 59.34],
    ['2022-03-28 00:00:00', '2022-10-31 00:00:00', 109.26],
    ['2023-03-27 00:00:00', '2023-10-30 00:00:00', 38.47],
]
data_cleaned = price_cleaner(info)

2019-04-01 00:00:00 2019-10-28 00:00:00
2020-03-30 00:00:00 2020-10-26 00:00:00
2021-03-29 00:00:00 2021-11-01 00:00:00
2022-03-28 00:00:00 2022-10-31 00:00:00
2023-03-27 00:00:00 2023-10-30 00:00:00


In [363]:
# data_cleaned.loc['2023-10-29 22:00:00': '2023-10-30 02:00:00']

,price,solar,wind_onshore,wind_offshore,total_load
2023-10-29 22:00:00,54.27,0,15805,6715,49518
2023-10-29 23:00:00,52.00,0,15103,6569,48285
2023-10-30 00:00:00,38.47,0,14484,6538,46748
2023-10-30 01:00:00,8.00,0,20160,6959,43967
2023-10-30 02:00:00,8.13,0,19183,6981,43740


In [364]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43824 entries, 2019-01-01 01:00:00 to 2024-01-01 00:00:00
Freq: h
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          43824 non-null  float64
 1   solar          43824 non-null  int64  
 2   wind_onshore   43824 non-null  int64  
 3   wind_offshore  43824 non-null  int64  
 4   total_load     43824 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 3.0 MB


In [365]:
data_cleaned.to_pickle('output/df_22_23_cleaned.pkl')